In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu bionic-se

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-20 06:22:44--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.6’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2022-06-20 06:22:46 (1.22 MB/s) - ‘postgresql-42.2.16.jar.6’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName("Amazon_Vine_Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show()
df.dtypes
df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

2402458

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(10)
vine_df.dtypes
vine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26MV8D0KG6QI6|          5|            0|          0|   N|                Y|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|                Y|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|                N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|                Y|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|                Y|
| RIG9AWFOGRDVO|          2|            1|          1|   N|                Y|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|                Y|
| RB15NBVY5ELVW|          5|            2|          2|   N|                Y|
| R56358YM1ZJ7I|          5|            0|          0|   N|                N|
|R1ODXB3C9UP3NL|          1|            1|          3|   N|     

2402458

**Determining Bias of Vine Reviews**

In [7]:
# Filtering the data to retrieve all rows where total_votes >= 20
# to pick reviews that are more likely to be helpful 

# Importing col function to more easily manipulate columns
from pyspark.sql.functions import col
# from pyspark.sql.types import IntegerType

high_votes_df = vine_df.where(vine_df.total_votes >= 20)


In [8]:
high_votes_df.show(10)
high_votes_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
|R33VIWRD2X5IA9|          1|            0|         20|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R26UFXLLQ2N0CR|          1|            0|         20|   N|                Y|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|     

31518

In [9]:
helpful_votes_df = high_votes_df.where(high_votes_df.helpful_votes / high_votes_df.total_votes > 0.5)
helpful_votes_df.show(10)
helpful_votes_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|                N|
| RO6JNMUJDE1LP|          5|           37|         50|   N|                N|
|R1WNL4OIMSKDBR|          5|           35|         36|   N|     

28235

In [10]:
paid_df = helpful_votes_df.where(helpful_votes_df.vine == 'Y')
unpaid_df = helpful_votes_df.where(helpful_votes_df.vine == 'N')

In [11]:
paid_df.show(10)
paid_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3MKO875WGIEBJ|          5|           34|         37|   Y|                N|
|R3OPNTK61FQ7MP|          5|           32|         37|   Y|                N|
| RMDRU8I773X5U|          4|           22|         25|   Y|                N|
| RR83RGWFEFZCL|          3|           61|         64|   Y|                N|
|R2TMZ7GV8SPIRV|          4|           23|         23|   Y|                N|
| RX88JLUIFA4Q2|          4|           84|         87|   Y|                Y|
| RV3AHTDRHDH7R|          5|          253|        258|   Y|                N|
|R13EGJI8C8YJ4Y|          5|           19|         20|   Y|                N|
| R9SNSQTK4TJOG|          2|           11|         20|   Y|                N|
|R1A76AK493C8P8|          5|           83|         90|   Y|     

61

In [12]:
unpaid_df.show(10)
unpaid_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|                N|
| RO6JNMUJDE1LP|          5|           37|         50|   N|                N|
|R1WNL4OIMSKDBR|          5|           35|         36|   N|     

28174

In [13]:
paid_info_df = spark.createDataFrame(
    [
      (paid_df.count(), paid_df.where(col("star_rating") == 5).count(), \
       paid_df.where(col("star_rating") == 5).count() / paid_df.count() \
      )
    ],
    ["paid_total_reviews", "5-star_reviews", "percent_5-star"]
)

paid_info_df.show()

+------------------+--------------+-------------------+
|paid_total_reviews|5-star_reviews|     percent_5-star|
+------------------+--------------+-------------------+
|                61|            20|0.32786885245901637|
+------------------+--------------+-------------------+



In [14]:
unpaid_info_df = spark.createDataFrame(
    [
      (unpaid_df.count(), unpaid_df.where(col("star_rating") == 5).count(), \
       unpaid_df.where(col("star_rating") == 5).count() / unpaid_df.count() \
      )
    ],
    ["unpaid_total_reviews", "5-star_reviews", "percent_5-star"]
)

unpaid_info_df.show()

+--------------------+--------------+------------------+
|unpaid_total_reviews|5-star_reviews|    percent_5-star|
+--------------------+--------------+------------------+
|               28174|         15672|0.5562575424149926|
+--------------------+--------------+------------------+

